# Create a Graph of the London Transport Network using LOAD CSV
Example Text:
In this notebook, let's explore how to leverage Google GenAI to build and consume a knowledge graph in Neo4j.

This notebook parses data from a public [corpus of Resumes / Curriculum Vitae](https://github.com/florex/resume_corpus) using Google Vertex AI Generative AI's `text-bison` model. The model will be prompted to recognise and extract entities and relationships. We will then generate Neo4j Cypher queries using them and write the data to a Neo4j database.
We will again use a `text-bison` model and prompt it to convert questions in english to Cypher - Neo4j's query language, which can be used for data retrieval.

## Setup
First off, check that the Python environment you installed in the readme is running this notebook. Make sure you select the `py38` kernel in the top right of this notebook. You should see a 3.8 version when you run this command.

In [ ]:
import sys
sys.version

Next we need to install some libraries.

In [ ]:
%pip install --user graphdatascience
%pip install --user neo4j
%pip install --user IProgress
%pip install --user tqdm

Now restart the kernel.  That will allow the Python evironment to import the new packages.

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## Establish Neo4j Connection

In [ ]:
# username is neo4j by default
NEO4J_USERNAME = 'neo4j'

# You will need to change these to match your credentials
NEO4J_URI = 'neo4j+s://a2a3a4e4.databases.neo4j.io'
NEO4J_PASSWORD = '7RbblizpZDpB_4INFovS75lSbHkAOcOJlG7KvZWyx84'

In [ ]:
from graphdatascience import GraphDataScience

gds = GraphDataScience(
    NEO4J_URI,
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD),
    aura_ds=True
)
gds.set_database('neo4j')
gds.run_cypher('RETURN gds.version()')

## Loading Data from CSV Files 

First let's load up the station nodes from a CSV file stored in a Cloud Storage bucket.

In [ ]:
gds.run_cypher('''
LOAD CSV WITH HEADERS FROM 'https://storage.googleapis.com/leerazo-demos/london_transport/datasets/London_stations.csv' AS row
MERGE (s:Station {latitude:toFloat(row.Latitude), longitude:toFloat(row.Longitude), name:row.Station, zone:row.Zone})
RETURN count(s) as stations
''')

Next let's connect those stations and label the connections according to tube lines.

In [ ]:
gds.run_cypher('''
LOAD CSV WITH HEADERS FROM 'https://storage.googleapis.com/leerazo-demos/london_transport/datasets/London_tube_lines.csv' as row
MATCH (a:Station), (b:Station) WHERE a.name = row.From_Station AND b.name = row.To_Station
CALL apoc.create.relationship(a, toUpper(row.Tube_Line), {}, b)
YIELD rel as rel1
CALL apoc.create.relationship(b, toUpper(row.Tube_Line), {}, a)
YIELD rel as rel2
RETURN count(rel1) + count(rel2) AS relationships;
''')

And now the graph is complete! You can delete the entire graph using the cell below to do it again or to move on to the next lab to try another data loading method. 